In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [4]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_TRACING_V2 = os.getenv("LANGSMITH_TRACING_V2")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [6]:
from langchain_tavily import TavilySearch

In [ ]:
search=TavilySearch(max_results=2)

In [23]:
def search_engine(query:str):
    """
    This function will be used to search web and get the relavent information based on the user query

    Agrs:
    query : user request
    """
    result=search.invoke(query)
    documents=[doc['content'] for doc in result['results']]
    final_doc="\n\n".join(documents)
    return final_doc
#result=search.invoke("what is the price of iphone 17 in india?")
search_engine("what is the price of iphone 17 in india?")

'iPhone 17 Pro Max Price in India. The iPhone 17 Pro Max starts at ₹1,49,900 for the 256GB variant and climbs to ₹2,29,900 for the 2TB model.\n\nThe iPhone 17 starts at Rs 82,900, while the iPhone 17 Pro and iPhone 17 Pro Max are priced at Rs 134,900 and Rs 149,900, respectively. These'

In [18]:
documents=[doc['content'] for doc in result['results']]
print("\n\n".join(documents))

iPhone 17 Pro Max Price in India. The iPhone 17 Pro Max starts at ₹1,49,900 for the 256GB variant and climbs to ₹2,29,900 for the 2TB model.

The iPhone 17 starts at Rs 82,900, while the iPhone 17 Pro and iPhone 17 Pro Max are priced at Rs 134,900 and Rs 149,900, respectively. These


In [10]:
def add(a:int,b:int):
    """
    This Function adds two integers

    Args:
    a : First integer
    b : Second integer
    """
    return a+b

def subtract(a:int,b:int):
    """
    This Function subtracts two integers

    Args:
    a : First integer
    b : Second integer
    """
    return a-b  

def Multiply(a:int,b:int):
    """
    This Function multiplies two integers

    Args:
    a : First integer
    b : Second integer
    """
    return a*b

def divide(a:int,b:int):
    """
    This Function divdes two integers

    Args:
    a : First integer
    b : Second integer
    """
    return a/b

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph,START,END

In [40]:
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash',temperature=0)
llm.invoke("what is the capital of india?")

AIMessage(content='The capital of India is **New Delhi**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--9e5a5679-ee43-43f7-b765-4c617c26c9a9-0', usage_metadata={'input_tokens': 8, 'output_tokens': 29, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 20}})

In [41]:
tool=[add,subtract,Multiply,divide,search_engine]
tool_node=ToolNode(tool)

In [42]:
llm_with_tools=llm.bind_tools(tool)

In [43]:
def tool_calling(state:MessagesState):
    response=llm_with_tools.invoke(state["messages"])
    return {"messages":[response]}


In [52]:
def router(state:MessagesState):
    last_message=state["messages"][-1]
    if last_message.tool_calls:
        return "tool"
    return END

In [53]:
builder=StateGraph(MessagesState)
builder.add_node("llm_with_tools",tool_calling)
builder.add_node("tool_store",tool_node)
builder.add_edge(START,"llm_with_tools")
builder.add_conditional_edges(
    "llm_with_tools",
    router,
    {
        "tool":"tool_store",
        END:END
    }
)
builder.add_edge("tool_store","llm_with_tools")
app=builder.compile()

In [76]:
state={"messages":["Hi what is your name?"]}
state1={"messages":["add 2 and 3"]}
state2={"messages":["Add 3 and 4. Multiply the output by 2. Divide the output by 5"]}
state3={"messages":["what is the price of iphone 17 in india"]}
state4={"messages":["what is the price difference between iphone 17 256 GB and 2TB"]}

In [55]:
app.get_graph().print_ascii()

           +-----------+              
           | __start__ |              
           +-----------+              
                 *                    
                 *                    
                 *                    
        +----------------+            
        | llm_with_tools |            
        +----------------+            
          ...         ***             
         .               *            
       ..                 **          
+---------+           +------------+  
| __end__ |           | tool_store |  
+---------+           +------------+  


In [56]:
response=app.invoke(state)

In [57]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Hi what is your name?
================================== Ai Message ==================================

I am a large language model, trained by Google.


In [58]:
response=app.invoke(state1)

In [59]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

add 2 and 3
================================== Ai Message ==================================
Tool Calls:
  add (e8c005d2-8e82-42f0-8840-9c6d6dd409d1)
 Call ID: e8c005d2-8e82-42f0-8840-9c6d6dd409d1
  Args:
    b: 3
    a: 2
================================= Tool Message =================================
Name: add

5
================================== Ai Message ==================================

The sum of 2 and 3 is 5.


In [60]:
response=app.invoke(state2)

In [61]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Add 3 and 4. Multiply the output by 2. Divide the output by 5
================================== Ai Message ==================================
Tool Calls:
  add (3c4d56e8-2727-4562-b15d-f2c3ef60be3b)
 Call ID: 3c4d56e8-2727-4562-b15d-f2c3ef60be3b
  Args:
    b: 4
    a: 3
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================
Tool Calls:
  Multiply (b48e17eb-1280-4543-a74e-542f6df96ab8)
 Call ID: b48e17eb-1280-4543-a74e-542f6df96ab8
  Args:
    b: 2
    a: 7
================================= Tool Message =================================
Name: Multiply

14
================================== Ai Message ==================================
Tool Calls:
  divide (4537899d-b861-4514-af71-1147ce24550a)
 Call ID: 4537899d-b861-4514-af71-1147ce24550a
  Args:
    b: 5
    a: 14
====================

In [74]:
response=app.invoke(state3)

In [75]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

what is the price of iphone 17 in india
================================== Ai Message ==================================
Tool Calls:
  search_engine (78043a1b-8010-46ec-a0dd-a7cd774cecec)
 Call ID: 78043a1b-8010-46ec-a0dd-a7cd774cecec
  Args:
    query: price of iphone 17 in india
================================= Tool Message =================================
Name: search_engine

iPhone 17 Pro Max Price in India. The iPhone 17 Pro Max starts at ₹1,49,900 for the 256GB variant and climbs to ₹2,29,900 for the 2TB model.

# List of 7 Countries Where iPhone 17 is More Affordable Than India List of Countries Where iPhone 17 is More Affordable Than India (Source Apple India) Apple products, especially the new model iPhones are often priced quite high in India in comparison to other countries. That is why the current pricing of the iPhone 17 in India starts from 82,900. Here is a look at a list of 7 countries w

In [77]:
response=app.invoke(state4)

In [78]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

what is the price difference between iphone 17 256 GB and 2TB
================================== Ai Message ==================================
Tool Calls:
  search_engine (bb7b2c59-2738-4d11-9add-c5501148a7a8)
 Call ID: bb7b2c59-2738-4d11-9add-c5501148a7a8
  Args:
    query: price of iphone 17 256 GB
  search_engine (e8a830d5-0b8c-49a9-99a6-ad9d2e228df3)
 Call ID: e8a830d5-0b8c-49a9-99a6-ad9d2e228df3
  Args:
    query: price of iphone 17 2TB
================================= Tool Message =================================
Name: search_engine

iPhone 17, Price, currently viewing. 256GB model from. $799. or $33.29/mo. for 24 mo.* Storage up to 512GB 6. Storage from 6. 256GB–512GB. iPhone Air, Price.

The baseline iPhone 17, with a 6.27-inch screen, starts at $799 with 256 GB of memory, the same price as the iPhone 16 with half the memory. The
================================= Tool Message ===================

In [79]:
response

{'messages': [HumanMessage(content='what is the price difference between iphone 17 256 GB and 2TB', additional_kwargs={}, response_metadata={}, id='f1699abf-1bc9-4dc2-b088-c7e6361333a7'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_engine', 'arguments': '{"query": "price of iphone 17 2TB"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--dcb93de6-e0b4-4c97-ae52-a1a817bda575-0', tool_calls=[{'name': 'search_engine', 'args': {'query': 'price of iphone 17 256 GB'}, 'id': 'bb7b2c59-2738-4d11-9add-c5501148a7a8', 'type': 'tool_call'}, {'name': 'search_engine', 'args': {'query': 'price of iphone 17 2TB'}, 'id': 'e8a830d5-0b8c-49a9-99a6-ad9d2e228df3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 274, 'output_tokens': 123, 'total_tokens': 397, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 75}})

In [83]:
state={"messages":[" i need to plan a trip to trivandrum,kerala. pls let me know the avg 4 start hotel price per day. convert the currency in INR. if i plan to stay for 5 days, how much it will cost me. i need trip itinery as well. pls summarize the entrie output"]}

In [84]:
response=app.invoke(state)

In [85]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

 i need to plan a trip to trivandrum,kerala. pls let me know the avg 4 start hotel price per day. convert the currency in INR. if i plan to stay for 5 days, how much it will cost me. i need trip itinery as well. pls summarize the entrie output
================================== Ai Message ==================================
Tool Calls:
  search_engine (5a8e5ece-4f21-41ac-9c8a-90d4e6528e35)
 Call ID: 5a8e5ece-4f21-41ac-9c8a-90d4e6528e35
  Args:
    query: average 4 star hotel price per day in Trivandrum INR
================================= Tool Message =================================
Name: search_engine

How much do 4-star hotels cost in Thiruvananthapuram? The average price of a 4-star hotel in Thiruvananthapuram is currently around ر.ق.‏ 142 per night. And

How much is a 4-star hotel in Trivandrum for this weekend? The average price per night for a 4-star hotel in Trivandrum this weekend is $52 (based 